<a href="https://colab.research.google.com/github/nedprz/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boosting/Ned_Przezdziecki_LS_DS_233_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
feb20 = pd.read_csv('drive/My Drive/2020_02.csv')
march20 = pd.read_csv('drive/My Drive/2020_03.csv')
march19 = pd.read_csv('drive/My Drive/201903-citibike-tripdata.csv')

In [0]:
march19

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,covid
0,1463,2019-03-01 00:00:16.097,2019-03-01 00:24:39.388,319.0,Fulton St & Broadway,40.711066,-74.009447,347.0,Greenwich St & W Houston St,40.728846,-74.008591,35618,Subscriber,1989,1,no
1,285,2019-03-01 00:00:32.385,2019-03-01 00:05:18.183,439.0,E 4 St & 2 Ave,40.726281,-73.989780,150.0,E 2 St & Avenue C,40.720874,-73.980858,31113,Subscriber,1980,1,no
2,686,2019-03-01 00:00:47.797,2019-03-01 00:12:14.309,526.0,E 33 St & 5 Ave,40.747659,-73.984907,3474.0,6 Ave & Spring St,40.725256,-74.004121,19617,Subscriber,1987,1,no
3,442,2019-03-01 00:01:01.209,2019-03-01 00:08:23.751,3474.0,6 Ave & Spring St,40.725256,-74.004121,355.0,Bayard St & Baxter St,40.716021,-73.999744,27086,Subscriber,1987,2,no
4,2913,2019-03-01 00:01:09.281,2019-03-01 00:49:42.381,379.0,W 31 St & 7 Ave,40.749156,-73.991600,212.0,W 16 St & The High Line,40.743349,-74.006818,34791,Subscriber,1991,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327955,351,2019-03-31 23:59:30.585,2019-04-01 00:05:21.785,3747.0,E 84 St & 3 Ave,40.777554,-73.955128,3351.0,E 102 St & 1 Ave,40.786995,-73.941648,37290,Subscriber,1985,1,no
1327956,1943,2019-03-31 23:59:55.341,2019-04-01 00:32:18.692,450.0,W 49 St & 8 Ave,40.762272,-73.987882,515.0,W 43 St & 10 Ave,40.760094,-73.994618,34220,Subscriber,1987,1,no
1327957,515,2019-03-31 23:59:50.181,2019-04-01 00:08:25.833,3542.0,Amsterdam Ave & W 119 St,40.808625,-73.959621,3543.0,Morningside Dr & Amsterdam Ave,40.810285,-73.957365,37863,Subscriber,1988,1,no
1327958,660,2019-03-31 23:59:56.652,2019-04-01 00:10:57.376,3434.0,W 88 St & West End Ave,40.790254,-73.977183,3538.0,W 110 St & Amsterdam Ave,40.802692,-73.962950,36103,Subscriber,1987,1,no


In [0]:
march20

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,covid
0,389,2020-03-01 00:14:07.000,2020-03-01 00:20:36.088,3202,Newport PATH,40.727224,-74.033759,3203,Hamilton Park,40.727596,-74.044247,42381,Subscriber,1992,1,yes
1,242,2020-03-01 00:48:06.439,2020-03-01 00:52:08.987,3185,City Hall,40.717733,-74.043845,3205,JC Medical Center,40.716540,-74.049638,42155,Subscriber,1991,1,yes
2,124,2020-03-01 01:08:46.488,2020-03-01 01:10:51.278,3272,Jersey & 3rd,40.723332,-74.045953,3278,Monmouth and 6th,40.725685,-74.048790,42376,Subscriber,1987,0,yes
3,104,2020-03-01 01:22:35.479,2020-03-01 01:24:20.247,3202,Newport PATH,40.727224,-74.033759,3638,Washington St,40.724294,-74.035483,42350,Subscriber,1993,1,yes
4,228,2020-03-01 01:39:20.268,2020-03-01 01:43:09.138,3194,McGinley Square,40.725340,-74.067622,3280,Astor Place,40.719282,-74.071262,42235,Subscriber,1988,1,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17714,296,2020-03-31 22:50:39.041,2020-03-31 22:55:35.598,3191,Union St,40.718211,-74.083639,3694,Jackson Square,40.711130,-74.078900,42145,Subscriber,1994,1,yes
17715,147,2020-03-31 23:09:15.969,2020-03-31 23:11:43.531,3640,Journal Square,40.733670,-74.062500,3206,Hilltop,40.731169,-74.057574,42110,Subscriber,1991,1,yes
17716,364,2020-03-31 23:17:02.024,2020-03-31 23:23:06.820,3195,Sip Ave,40.730897,-74.063913,3679,Bergen Ave,40.722104,-74.071455,42550,Subscriber,1986,1,yes
17717,1217,2020-03-31 23:36:10.643,2020-03-31 23:56:27.812,3267,Morris Canal,40.712419,-74.038526,3276,Marin Light Rail,40.714584,-74.042817,42115,Subscriber,1986,1,yes


In [0]:
feb20

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,841,2020-02-01 00:02:13.7650,2020-02-01 00:16:15.4720,3199,Newport Pkwy,40.728745,-74.032108,3213,Van Vorst Park,40.718489,-74.047727,42217,Subscriber,1980,1
1,206,2020-02-01 00:16:11.3850,2020-02-01 00:19:37.4460,3273,Manila & 1st,40.721651,-74.042884,3639,Harborside,40.719252,-74.034234,42160,Subscriber,1990,1
2,859,2020-02-01 00:23:22.1090,2020-02-01 00:37:41.5060,3185,City Hall,40.717733,-74.043845,3268,Lafayette Park,40.713464,-74.062859,42204,Customer,1991,1
3,693,2020-02-01 00:25:14.5170,2020-02-01 00:36:48.0280,3185,City Hall,40.717733,-74.043845,3277,Communipaw & Berry Lane,40.714358,-74.066611,42165,Subscriber,1987,1
4,1473,2020-02-01 00:32:30.5050,2020-02-01 00:57:04.3740,3192,Liberty Light Rail,40.711242,-74.055701,3639,Harborside,40.719252,-74.034234,26156,Customer,1969,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22957,262,2020-02-29 23:40:53.1620,2020-02-29 23:45:15.3180,3639,Harborside,40.719252,-74.034234,3214,Essex Light Rail,40.712774,-74.036486,42467,Subscriber,1967,1
22958,216,2020-02-29 23:41:01.3350,2020-02-29 23:44:38.0350,3186,Grove St PATH,40.719586,-74.043117,3483,Montgomery St,40.719420,-74.050990,42377,Subscriber,1958,1
22959,101,2020-02-29 23:45:22.1700,2020-02-29 23:47:03.2970,3214,Essex Light Rail,40.712774,-74.036486,3681,Grand St,40.715178,-74.037683,42485,Subscriber,1967,1
22960,279,2020-02-29 23:47:56.8380,2020-02-29 23:52:36.6400,3186,Grove St PATH,40.719586,-74.043117,3483,Montgomery St,40.719420,-74.050990,42531,Subscriber,1988,1


In [0]:
march20['starttime']= pd.to_datetime(march20['starttime']) 
march20['stoptime']= pd.to_datetime(march20['stoptime']) 
march19['starttime']= pd.to_datetime(march19['starttime']) 
march19['stoptime']= pd.to_datetime(march19['stoptime']) 
march20.dtypes

tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int64
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int64
usertype                           object
birth year                          int64
gender                              int64
dtype: object

In [0]:
march20['covid']='yes'
march19['covid']='no'

In [0]:
march19and20 = pd.concat([march19, march20], ignore_index=True)
march19and20

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,covid
0,1463,2019-03-01 00:00:16.097,2019-03-01 00:24:39.388,319.0,Fulton St & Broadway,40.711066,-74.009447,347.0,Greenwich St & W Houston St,40.728846,-74.008591,35618,Subscriber,1989,1,no
1,285,2019-03-01 00:00:32.385,2019-03-01 00:05:18.183,439.0,E 4 St & 2 Ave,40.726281,-73.989780,150.0,E 2 St & Avenue C,40.720874,-73.980858,31113,Subscriber,1980,1,no
2,686,2019-03-01 00:00:47.797,2019-03-01 00:12:14.309,526.0,E 33 St & 5 Ave,40.747659,-73.984907,3474.0,6 Ave & Spring St,40.725256,-74.004121,19617,Subscriber,1987,1,no
3,442,2019-03-01 00:01:01.209,2019-03-01 00:08:23.751,3474.0,6 Ave & Spring St,40.725256,-74.004121,355.0,Bayard St & Baxter St,40.716021,-73.999744,27086,Subscriber,1987,2,no
4,2913,2019-03-01 00:01:09.281,2019-03-01 00:49:42.381,379.0,W 31 St & 7 Ave,40.749156,-73.991600,212.0,W 16 St & The High Line,40.743349,-74.006818,34791,Subscriber,1991,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345674,296,2020-03-31 22:50:39.041,2020-03-31 22:55:35.598,3191.0,Union St,40.718211,-74.083639,3694.0,Jackson Square,40.711130,-74.078900,42145,Subscriber,1994,1,yes
1345675,147,2020-03-31 23:09:15.969,2020-03-31 23:11:43.531,3640.0,Journal Square,40.733670,-74.062500,3206.0,Hilltop,40.731169,-74.057574,42110,Subscriber,1991,1,yes
1345676,364,2020-03-31 23:17:02.024,2020-03-31 23:23:06.820,3195.0,Sip Ave,40.730897,-74.063913,3679.0,Bergen Ave,40.722104,-74.071455,42550,Subscriber,1986,1,yes
1345677,1217,2020-03-31 23:36:10.643,2020-03-31 23:56:27.812,3267.0,Morris Canal,40.712419,-74.038526,3276.0,Marin Light Rail,40.714584,-74.042817,42115,Subscriber,1986,1,yes


In [0]:
target = 'covid'
features = 